# Premier League Web Crawler Pt. 1

In [1]:
#IMPORT LIBRARIEs
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
#Create the spider class
class premSpider(scrapy.Spider):
    
    name = "premSpider"
    
    custom_settings = {'FEEDS':{'results.csv':{'format':'csv'}}}
    
    def start_requests(self):
        
        # Create a List of Urls with which we wish to scrape
        urls = ['https://www.premierleague.com/players']
        
        #Iterate through each url and send it to be parsed
        
        for url in urls:
            
            #yield kind of acts like return
            yield scrapy.Request(url = url, callback = self.parse)
            
    def parse(self, response):
        
        #extract links to player pages
        plinks = response.xpath('//tr').css('a::attr(href)').extract()
        
        #follow links to specific player pages
        for plink in plinks:
            
            yield response.follow(url = plink, callback = self.parse2)
            
    def parse2(self, response):
        
        plinks2 = response.xpath('//a[@href="stats"]').css('a::attr(href)').extract()
        
        for link2 in plinks2:
            
            yield response.follow(url = link2, callback = self.parse3)
        
    def parse3(self, response):
        
        names = response.xpath('.//div[@class="name t-colour"]/text()').extract()
        appearances = response.xpath('//span[@data-stat="appearances"]//text()').extract_first().strip()
          
        atkstat = []
        for val in response.xpath('//div[@class="normalStat"]/span/span//text()').extract():
            atk = val.strip()
            atkstat.append(atk)
            
            
        statname = []
        for val in response.xpath('//div[@class="normalStat"]/span[@class="stat"]/text()').extract():
            if val != '\n':
                statname.append(val.strip())
        
        stat_dict = dict(zip(statname, atkstat))
                

        
        yield {'Names': names, 'Appearances': appearances, **stat_dict}
        

#initiate crawler process
process = CrawlerProcess()

#Tell the process which spider to use
process.crawl(premSpider)


#start the crawling process
process.start()       
        


2020-12-12 22:38:46 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-12 22:38:46 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-12-12 22:38:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-12-12 22:38:46 [scrapy.crawler] INFO: Overridden settings:
{}
2020-12-12 22:38:46 [scrapy.extensions.telnet] INFO: Telnet Password: 1ba33ced3e26e172
2020-12-12 22:38:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-12-12 22:38:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermi